In [1]:
from numpy import int64
import numpy as np
import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime

In [2]:
Oct_stacked=pd.read_csv("Oct_data.tsv",sep="\t",low_memory=False)
Oct_stacked.columns.sort_values(ascending=False)

Index(['id_physician', 'Medialength.9', 'Medialength.8', 'Medialength.7',
       'Medialength.6', 'Medialength.5', 'Medialength.4', 'Medialength.3',
       'Medialength.29', 'Medialength.28',
       ...
       'Appointments.10', 'Appointments.1', 'Appointments.0',
       'Appointments.-7', 'Appointments.-6', 'Appointments.-5',
       'Appointments.-4', 'Appointments.-3', 'Appointments.-2',
       'Appointments.-1'],
      dtype='object', length=112)

In [3]:
Oct_stacked.dropna(subset="EncounterDate.empty",inplace=True)
Oct_stacked=Oct_stacked[Oct_stacked["EncounterDate.empty"]!="empty"]
Oct_stacked.columns

Index(['id_physician', 'Encounters.4', 'Encounters.6', 'Encounters.7',
       'Encounters.13', 'Encounters.14', 'Encounters.18', 'Encounters.20',
       'Encounters.21', 'Encounters.25',
       ...
       'Appointments.15', 'Appointments.23', 'Encounters.-6', 'Encounters.-5',
       'Medialength.-6', 'Medialength.-5', 'Encounters.9', 'Medialength.9',
       'Appointments.9', 'Current_Day.empty'],
      dtype='object', length=112)

In [4]:
predicted_encounters=['%s.%s'%("Encounters",day)
                     for day in range(-1,-8,-1)]
predicted_medialength= ['%s.%s'%("Medialength",day)
                     for day in range(-1,-8,-1)]
predicted_encounters_in_Oct_stacked=[column for column in predicted_encounters if column in Oct_stacked]


In [5]:
predicted_encounters,predicted_medialength

(['Encounters.-1',
  'Encounters.-2',
  'Encounters.-3',
  'Encounters.-4',
  'Encounters.-5',
  'Encounters.-6',
  'Encounters.-7'],
 ['Medialength.-1',
  'Medialength.-2',
  'Medialength.-3',
  'Medialength.-4',
  'Medialength.-5',
  'Medialength.-6',
  'Medialength.-7'])

In [6]:
train_pred=Oct_stacked[pd.to_datetime(Oct_stacked["EncounterDate.empty"]).dt.date<datetime.date(2022,10,22)]
test_pred= Oct_stacked[pd.to_datetime(Oct_stacked["EncounterDate.empty"]).dt.date>datetime.date(2022,10,22)]

X_train_pred =train_pred.drop(columns= predicted_encounters + predicted_medialength,errors="ignore")
X_train_pred= X_train_pred.drop(['EncounterDate.empty'], axis=1)
Y_train_pred_7=train_pred[predicted_encounters_in_Oct_stacked]
X_test_pred=test_pred.drop(columns=predicted_encounters + predicted_medialength,errors="ignore")
X_test_pred= X_test_pred.drop(['EncounterDate.empty'], axis=1)
Y_test_pred_7=test_pred[predicted_encounters_in_Oct_stacked]
# Oct_stacked.columns
X_test_pred.columns

Index(['id_physician', 'Encounters.4', 'Encounters.6', 'Encounters.7',
       'Encounters.13', 'Encounters.14', 'Encounters.18', 'Encounters.20',
       'Encounters.21', 'Encounters.25', 'Encounters.27', 'Encounters.28',
       'Medialength.4', 'Medialength.6', 'Medialength.7', 'Medialength.13',
       'Medialength.14', 'Medialength.18', 'Medialength.20', 'Medialength.21',
       'Medialength.25', 'Medialength.27', 'Medialength.28', 'Appointments.-7',
       'Appointments.-3', 'Appointments.-1', 'Appointments.0',
       'Appointments.4', 'Appointments.6', 'Appointments.7', 'Appointments.13',
       'Appointments.14', 'Appointments.18', 'Appointments.20',
       'Appointments.21', 'Appointments.25', 'Appointments.27',
       'Appointments.28', 'Encounters.5', 'Encounters.11', 'Medialength.5',
       'Medialength.11', 'Appointments.-4', 'Appointments.5',
       'Appointments.11', 'Encounters.3', 'Encounters.10', 'Encounters.12',
       'Encounters.17', 'Encounters.19', 'Encounters.26', '

In [7]:
print(predicted_encounters_in_Oct_stacked)

['Encounters.-1', 'Encounters.-2', 'Encounters.-3', 'Encounters.-4', 'Encounters.-5', 'Encounters.-6', 'Encounters.-7']


In [8]:
def random_forest_day(X_train_pred,Y_train_pred_7,X_test_pred,Y_test_pred_7,day):

    Enc_day="Encounters.%d"%day
    LX_train_pred=X_train_pred.copy()
    LX_train_pred.fillna(value=0,inplace=True)

    LX_test_pred=X_test_pred.copy()
    LX_test_pred.fillna(value=0,inplace=True)

    LY_train_pred=Y_train_pred_7[Enc_day].copy()

    LY_train_pred.columns=[day]
    LY_train_pred.fillna(value=0,inplace=True)
    
    LY_test_pred=Y_test_pred_7.copy()
    LY_test_pred.fillna(value=0,inplace=True)

    model = RandomForestRegressor(n_estimators=100,
                                  criterion='squared_error',
                                  random_state=None,
                                  max_samples= None,
                                  max_features= 0.8 ,
                                  max_depth=60)

    model.fit(LX_train_pred, LY_train_pred)
    print(LX_test_pred)
    y_pred_RF = model.predict(LX_test_pred)
    y_pred_RF_df=pd.DataFrame({Enc_day+"_predicted":y_pred_RF},index=LX_test_pred.index)
    df_result=pd.concat([LX_test_pred,LY_test_pred,y_pred_RF_df],axis=1,join = 'outer')

    return df_result

In [9]:
Result_day1=random_forest_day(X_train_pred,Y_train_pred_7,X_test_pred,Y_test_pred_7,day=-1)

       id_physician  Encounters.4  Encounters.6  Encounters.7  Encounters.13  \
36407           165           0.0          50.0           0.0           45.0   
36408           676           0.0          40.0           0.0           43.0   
36409           678          43.0          33.0           0.0            0.0   
36410           689          27.0           1.0           0.0           29.0   
36411           702           0.0          42.0           0.0           39.0   
...             ...           ...           ...           ...            ...   
56285         73060           0.0           0.0           0.0            0.0   
56286         73062           0.0           0.0           0.0            0.0   
56287         73068           0.0           0.0           0.0            0.0   
56288         73089           0.0           0.0           0.0            0.0   
56289         73147           0.0           0.0           0.0            0.0   

       Encounters.14  Encounters.18  En

In [10]:
X_train_pred.shape,Y_train_pred_7.shape,X_test_pred.shape,Y_test_pred_7.shape,Result_day1.shape

((33576, 97), (33576, 7), (19883, 97), (19883, 7), (19883, 105))

In [11]:
Result_day1.tail(8)

,id_physician,Encounters.4,Encounters.6,Encounters.7,Encounters.13,Encounters.14,Encounters.18,Encounters.20,Encounters.21,Encounters.25,...,Appointments.9,Current_Day.empty,Encounters.-1,Encounters.-2,Encounters.-3,Encounters.-4,Encounters.-5,Encounters.-6,Encounters.-7,Encounters.-1_predicted
56282,73051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.16
56283,73053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04
56284,73055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12
56285,73060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.16
56286,73062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.16
56287,73068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12
56288,73089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12
56289,73147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12


In [12]:
print(Result_day1.isnull().values.any())

False


In [13]:
print(Result_day1.columns)

Index(['id_physician', 'Encounters.4', 'Encounters.6', 'Encounters.7',
       'Encounters.13', 'Encounters.14', 'Encounters.18', 'Encounters.20',
       'Encounters.21', 'Encounters.25',
       ...
       'Appointments.9', 'Current_Day.empty', 'Encounters.-1', 'Encounters.-2',
       'Encounters.-3', 'Encounters.-4', 'Encounters.-5', 'Encounters.-6',
       'Encounters.-7', 'Encounters.-1_predicted'],
      dtype='object', length=105)


In [14]:
#Evaluate per day
def Oct_test_day(Y_test_pred_7,day):
        
    Enc_day="Encounters.%d"%day
    LY_test_pred= Y_test_pred_7[Enc_day].copy()
    LY_test_pred.columns=[day]
    LY_test_pred.fillna(value=0,inplace=True)
    return LY_test_pred

In [15]:
y_test_pred_day_M1=Oct_test_day(Y_test_pred_7,day=-1)
y_test_pred_day_M7=Oct_test_day(Y_test_pred_7,day=-7)

In [16]:
def evaluate(Y_test_pred, y_pred_RF):
        print("Type of Y_test_pred:",type(Y_test_pred))
        print("Type of y_pred_RF:",type(y_pred_RF))
        print("Y_test_pred null value:",Y_test_pred.isnull().values.any())
#       print("y_pred_RF null value:",y_pred_RF.isnull().values.any())
        print("y_pred_RF null value:",sum(np.isnan(y_pred_RF)))
        mse_RF = mean_squared_error(Y_test_pred, y_pred_RF)
        mae_RF = mean_absolute_error(Y_test_pred, y_pred_RF)
        R_sqr_score=r2_score(Y_test_pred,y_pred_RF)
        print('Mean squared error using Random Forest: ', mse_RF)
        print('Mean absolute error using Random Forest: ', mae_RF)
        print('R2 score:',R_sqr_score) 
        
        return mse_RF,mae_RF,R_sqr_score

In [17]:
evaluate(Result_day1.iloc[:,-2],Result_day1.iloc[:,-1])
# evaluate(y_test_pred_day_M7[-2],Result_day1[-1])

# print(y_test_pred_day_M7.query('Encounters.-1'==0).shape)
# print(y_pred_RF_list[0].query('Encounters.-1'==0).shape)
# sum(y_test_pred_day_M1==0)
# print(sum(y_test_pred_day_M1==0))
# print(sum(y_pred_RF_df<0.5))
# print(y_pred_RF_df.shape)

Type of Y_test_pred: <class 'pandas.core.series.Series'>
Type of y_pred_RF: <class 'pandas.core.series.Series'>
Y_test_pred null value: False
y_pred_RF null value: 0
Mean squared error using Random Forest:  59.599327817733744
Mean absolute error using Random Forest:  3.9277774983654385
R2 score: -2.690798398805193


(59.599327817733744, 3.9277774983654385, -2.690798398805193)

In [18]:
type(y_test_pred_day_M7)

pandas.core.series.Series

In [19]:
# sns.jointplot(x=y_test_pred_day_M7,y=y_pred_RF_dict[-7])

NameError: name 'y_pred_RF_dict' is not defined

In [20]:
# sns.jointplot(x=y_test_pred_day_M7,y=y_pred_RF_dict[-1])

In [21]:
X_test_pred.columns.sort_values()

Index(['Appointments.-1', 'Appointments.-2', 'Appointments.-3',
       'Appointments.-4', 'Appointments.-5', 'Appointments.-6',
       'Appointments.-7', 'Appointments.0', 'Appointments.1',
       'Appointments.10', 'Appointments.11', 'Appointments.12',
       'Appointments.13', 'Appointments.14', 'Appointments.15',
       'Appointments.16', 'Appointments.17', 'Appointments.18',
       'Appointments.19', 'Appointments.2', 'Appointments.20',
       'Appointments.21', 'Appointments.22', 'Appointments.23',
       'Appointments.24', 'Appointments.25', 'Appointments.26',
       'Appointments.27', 'Appointments.28', 'Appointments.29',
       'Appointments.3', 'Appointments.4', 'Appointments.5', 'Appointments.6',
       'Appointments.7', 'Appointments.8', 'Appointments.9',
       'Current_Day.empty', 'Encounters.1', 'Encounters.10', 'Encounters.11',
       'Encounters.12', 'Encounters.13', 'Encounters.14', 'Encounters.15',
       'Encounters.16', 'Encounters.17', 'Encounters.18', 'Encounters.

In [22]:
def evaluate_subset(Xyp,query_str,day):

    
    Xyp_subset=Xyp.query(query_str).copy()
    print("Xyp subset:",Xyp_subset.isnull().values.any())
    print("Xyp_shape:",Xyp_subset.shape)
    print("Test for infinity:" ,Xyp_subset[np.isfinite(Xyp_subset).all(1)])
    
    Xyp_subset.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("Test for infinity:" ,Xyp_subset[np.isfinite(Xyp_subset).all(1)])
    print("Xyp y_pred_RF:",Xyp["y_pred_RF_df"].to_numpy())
    print("Y_test_pred:",Xyp["Y_test_pred_df"])
   
    evaluate(Xyp.iloc[:,-2],Xyp.iloc[:,-1])
    
  
             
 #concatinate above 3 dataframe into new dataframe(Xyp)----> done 

In [23]:
evaluate_subset(Result_day1,query_str="id_physician==id_physician",day=-1)

Xyp subset: False
Xyp_shape: (19883, 105)
Test for infinity:        id_physician  Encounters.4  Encounters.6  Encounters.7  Encounters.13  \
36407           165           0.0          50.0           0.0           45.0   
36408           676           0.0          40.0           0.0           43.0   
36409           678          43.0          33.0           0.0            0.0   
36410           689          27.0           1.0           0.0           29.0   
36411           702           0.0          42.0           0.0           39.0   
...             ...           ...           ...           ...            ...   
56285         73060           0.0           0.0           0.0            0.0   
56286         73062           0.0           0.0           0.0            0.0   
56287         73068           0.0           0.0           0.0            0.0   
56288         73089           0.0           0.0           0.0            0.0   
56289         73147           0.0           0.0           0

KeyError: 'y_pred_RF_df'

In [25]:
def compute_appointment_only_baseline(X_test_pred,Y_test_pred_7,day):
    Enc_day="Encounters.%d"%day
    App_day="Appointments.%d"%day
    One_App=X_test_pred[App_day].fillna(0)
    Onc_Enc=Y_test_pred_7[Enc_day].fillna(0)
    df_result=pd.concat([X_test_pred,Onc_Enc,One_App],axis=1,join = 'outer')

    return df_result

In [26]:
Appointment_only_day1=compute_appointment_only_baseline(X_test_pred,Y_test_pred_7,day=-1)
Appointment_only_day7=compute_appointment_only_baseline(X_test_pred,Y_test_pred_7,day=-7)

In [27]:
evaluate_subset(Appointment_only_day1,query_str="id_physician==id_physician",day=-1)
evaluate_subset(Appointment_only_day7,query_str="id_physician==id_physician",day=-7)

Xyp subset: True
Xyp_shape: (19883, 99)
Test for infinity: Empty DataFrame
Columns: [id_physician, Encounters.4, Encounters.6, Encounters.7, Encounters.13, Encounters.14, Encounters.18, Encounters.20, Encounters.21, Encounters.25, Encounters.27, Encounters.28, Medialength.4, Medialength.6, Medialength.7, Medialength.13, Medialength.14, Medialength.18, Medialength.20, Medialength.21, Medialength.25, Medialength.27, Medialength.28, Appointments.-7, Appointments.-3, Appointments.-1, Appointments.0, Appointments.4, Appointments.6, Appointments.7, Appointments.13, Appointments.14, Appointments.18, Appointments.20, Appointments.21, Appointments.25, Appointments.27, Appointments.28, Encounters.5, Encounters.11, Medialength.5, Medialength.11, Appointments.-4, Appointments.5, Appointments.11, Encounters.3, Encounters.10, Encounters.12, Encounters.17, Encounters.19, Encounters.26, Medialength.3, Medialength.10, Medialength.12, Medialength.17, Medialength.19, Medialength.26, Appointments.3, Appoi

KeyError: 'y_pred_RF_df'

In [28]:
def compute_max_baseline(X_test_pred,Y_test_pred_7,day):
    
    df_result=pd.concat([LX_test_pred,LY_test_pred,y_pred_RF_df],axis=1,join = 'outer')

    return df_result

In [29]:
max_preds_day_M1=compute_max_baseline(X_test_pred,Y_test_pred_7,day=-1)
max_preds_day_M7=compute_max_baseline(X_test_pred,Y_test_pred_7,day=-7)

NameError: name 'LX_test_pred' is not defined

In [24]:
evaluate_subset(max_preds_day_M1,query_str="id_physician==id_physician",day=-1)
evaluate_subset(max_preds_day_M7,query_str="id_physician==id_physician",day=-7)

NameError: name 'max_preds_day_M1' is not defined